In [ ]:
# TRAINING ENSEMBLE MULTI-MODELS
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.ensemble_model import run_ensemble_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Prepare data generators
train_generator, validation_generator, test_generator, label_encoder, class_weights = get_data_generators()
# Run the ensemble model
run_ensemble_model(train_generator, validation_generator, test_generator, class_weights)

In [ ]:
# TRAINING SINGLE EFFICIENTNETB3 MODEL - VERSION 1
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.efficientnet_model import train_efficientnetb3, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_gen, val_gen, test_gen, label_enc, class_wts = get_data_generators()

# Train EfficientNetB3
model, history = train_efficientnetb3(train_gen, val_gen, class_wts)

# Evaluate on Test Set
evaluate_model(model, test_gen)


In [ ]:
# TRAINING SINGLE EFFICIENTNETB3 MODEL - VERSION 2
import sys
import pandas as pd
from utils.preprocess_2 import get_data_generators
from ai_models.efficientnet_model import train_efficientnetb3, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_gen, val_gen, test_gen, class_wts = get_data_generators()

# Train EfficientNetB3
model, history = train_efficientnetb3(train_gen, val_gen, class_wts)

# Evaluate on Test Set
evaluate_model(model, test_gen)

In [ ]:
# TRAINING SINGLE CUSTOM-CNN MODEL
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.cnn_model import train_custom_cnn, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_generator, validation_generator, test_generator, label_encoder, class_weights = get_data_generators()

# Train the model
model, history = train_custom_cnn(train_generator, validation_generator)

# Evaluate on validation set
evaluate_model(model, test_generator)

In [ ]:
# CHECKING ENSEMBLE MULTI-MODEL PREDICTIONS
import sys
import numpy as np
from tensorflow.keras.models import load_model
from joblib import load
from tensorflow.keras.preprocessing.image import load_img, img_to_array

sys.path.append(r'D:\skin_disease_detection\trained_models')

# Load base models
resnet_model = load_model(r"D:\skin_disease_detection\trained_models\ResNet50_model.h5")
print("ResNet50 model loaded successfully!")
efficientnet_model = load_model(r"D:\skin_disease_detection\trained_models\EfficientNetB0_model.h5")
print("EfficientNet model loaded successfully!") 
densenet_model = load_model(r"D:\skin_disease_detection\trained_models\DenseNet121_model.h5")
print("DenseNet model loaded successfully!")

# Load meta-model
meta_model = load(r"D:\skin_disease_detection\trained_models\meta_model.pkl")
print("Meta-model loaded successfully!")

# Load an image
image_path = r"D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_2\ISIC_0029360.jpg"
img = load_img(image_path, target_size=(224, 224))  # Resize as per model input size
print(img)

# Preprocess the image
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Convert single image to batch
img_array = img_array / 255.0 

# Get predictions from the base models
resnet_pred = resnet_model.predict(img_array)
efficientnet_pred = efficientnet_model.predict(img_array)
densenet_pred = densenet_model.predict(img_array)

print("ResNet50 Prediction:", resnet_pred)
print("EfficientNet Prediction:", efficientnet_pred)
print("DenseNet Prediction:", densenet_pred)

# Combine predictions for the meta-model
base_predictions = np.array([resnet_pred, efficientnet_pred, densenet_pred]).reshape(1, -1)

# Get the final prediction from the meta-model
meta_prediction = meta_model.predict(base_predictions)
print("Meta-Model Final Prediction:", meta_prediction)

In [ ]:
# CHECKING SINGLE-MODEL EFICIENTNETB3 PREDICTIONS
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

# Load the trained model
MODEL_PATH = r'D:\skin_disease_detection\trained_models\efficientnetb3.h5'  # Update as needed
model = tf.keras.models.load_model(MODEL_PATH)

# Define class labels (ensure these match your training labels)
class_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']  # Confirm correct order

def preprocess_image(image_path, target_size=(224, 224)):
    """Preprocess an image for model prediction."""
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize
    return img_array

def predict_skin_disease(image_path):
    """Predict skin disease and print detailed output."""
    img_array = preprocess_image(image_path)
    
    # Get raw predictions
    predictions = model.predict(img_array)
    
    # Print softmax probabilities for debugging
    print("\n########## SOFTMAX PROBABILITIES ##########")
    for i, prob in enumerate(predictions[0]):
        print(f"{class_labels[i]}: {prob:.4f}")

    # Get most likely class
    predicted_class = np.argmax(predictions)  
    confidence = np.max(predictions)  

    # Map to class label
    predicted_label = class_labels[predicted_class]

    print(f"\nPredicted Class: {predicted_label} (Confidence: {confidence:.2f})")

# Test with a real image
TEST_IMAGE_PATH = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_1\ISIC_0027419.jpg'  # Update as needed
predict_skin_disease(TEST_IMAGE_PATH)
